In [ ]:
%pip install pandas
%pip install seaborn
%pip install matplotlib
%pip install sklearn
import pickle as pkl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [22]:
%pip install scikit-learn
%pip install fsspec
import sklearn as sk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 154.0/154.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
# load pickle into dataframe
df = pd.DataFrame()
with open("C://Users/testuser/Documents/manningProjects/MeasuringBias/mortgage_data_project_one.pkl", 'rb') as f:
    df = pd.read_pickle(f)

# keep only the unique cols
keep_vars = ['agency_abbr', 'loan_type_name', 'loan_amount_000s', 'owner_occupancy_name', 'loan_purpose_name', 'property_type_name', 'applicant_ethnicity_name', 'applicant_race_name_1', 'applicant_sex_name', 'applicant_income_000s', 'population', 'minority_population', 'hud_median_family_income', 'tract_to_msamd_income', 'number_of_owner_occupied_units', 'number_of_1_to_4_family_units', 'action_taken_name']

df = df[keep_vars]

In [29]:
cat_variables = ['applicant_ethnicity_name', 'applicant_race_name_1', 'applicant_sex_name', 'agency_abbr', 'owner_occupancy_name', 'property_type_name', 'loan_purpose_name', 'loan_type_name']

# convert categorical variables to dummy variables using OneHotEncoder
# https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df[cat_variables]).toarray())
# get feature names from OneHotEncoder
enc_df.columns = enc.get_feature_names_out(cat_variables)
# join the encoded df
df = df.join(enc_df)
df = df.drop(cat_variables, axis=1)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165950 entries, 0 to 165949
Data columns (total 36 columns):
 #   Column                                                                            Non-Null Count   Dtype  
---  ------                                                                            --------------   -----  
 0   loan_amount_000s                                                                  165950 non-null  int64  
 1   applicant_income_000s                                                             165950 non-null  float64
 2   population                                                                        165950 non-null  float64
 3   minority_population                                                               165950 non-null  float64
 4   hud_median_family_income                                                          165950 non-null  float64
 5   tract_to_msamd_income                                                             165950 non-null  f

In [39]:
# run checks using assert statements
# https://realpython.com/python-assert-statement/

# assert the there are 27 columns in enc_df
assert enc_df.columns.size == 27, f"27 categorical variables expected, got: {enc_df.columns.size}"

# assert that the number of rows in df is the same as the number of rows in enc_df
assert df.shape[0] == 165950, f"165950 rows expected, got: {df.shape[0]}"


165950

In [41]:
df['action_taken_name']

# convert action_taken_name to True if "Application denied by financial institution" and False otherwise
df['action_taken_name'] = df['action_taken_name'].apply(lambda x: True if x == 'Application denied by financial institution' else False)


In [45]:
# count the number of True and False values in action_taken_name
df['action_taken_name'].value_counts()

# assert the shape of df
assert df.shape == (165950, 36), f"df shape expected: (165950, 36), got: {df.shape}"

In [47]:
# save df to pickle
with open("./mortgage_data_preprocessed.pkl", 'wb') as f:
    df.to_pickle(f)
